# 데이터베이스 내에 넣을 데이터프레임 가공

# 순서 - master_crop_variety -> map_region_weather_station -> weather_daily -> factor_external -> fact_trade

In [73]:
import pandas as pd
# 외생요소 가공
df_factor_external = pd.read_csv('test/holiday_작기2.csv', encoding='utf-8')

# 멜팅
df_melted = df_factor_external.melt(
    id_vars=['date', 'holiday_flag', 'holiday_score'],  # 고정 컬럼
    var_name='crop_name',         # 새로 생길 품목명 컬럼명
    value_name='grow_score'       # 각 품목의 작기지수 값 컬럼명
)

# 3. 필요하면 중복 제거
df_melted = df_melted.drop_duplicates()

# 4. 필요하면 인덱스 리셋
df_melted = df_melted.reset_index(drop=True)

print(df_melted.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20781 entries, 0 to 20780
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           20781 non-null  object 
 1   holiday_flag   20781 non-null  int64  
 2   holiday_score  20781 non-null  float64
 3   배추             20779 non-null  float64
 4   과수             20779 non-null  float64
 5   양파             20779 non-null  float64
 6   상추             20779 non-null  float64
 7   무              20779 non-null  float64
 8   마늘             20779 non-null  float64
 9   건고추            20779 non-null  float64
 10  감자             20779 non-null  float64
dtypes: float64(9), int64(1), object(1)
memory usage: 1.7+ MB


In [75]:
# 품목
# 1. 품목명 → 품목코드 매핑 딕셔너리 수동 작성
crop_code_map = {
    "양파": "1201",  
    "배추": "1001", 
    "상추": "1005",  
    "과수": "0601",   
    "무" : "1101",
#    '배' : '0602',
    "마늘" : "1209",
    "건고추" : "1207",
    "감자" : "0501",
}


# 2. crop_name → crop_code로 변환 컬럼 추가
df_melted['item_code'] = df_melted['crop_name'].map(crop_code_map)

In [76]:
df_melted.to_csv('test/holiday_작기_수정완료_utf-8.csv', encoding='utf-8', index=False)

In [85]:
df = pd.read_csv('test/holiday_작기_수정완료_utf-8.csv', encoding='utf-8')

In [108]:
# 주간 병합
from datetime import datetime, date

def get_week_of_year(date):
    date = pd.to_datetime(date)
    year = date.year
    week_number = date.isocalendar().week
    return f"{year}년 {week_number:02d}째주"

df['week_no'] = df['date'].apply(get_week_of_year)
df.drop(columns=['date', 'crop_name'], inplace=True)
df['item_code'] = df['item_code'].astype(str)
# 2자리 숫자로 패딩 (예: '1' -> '01')
df['item_code']  = df['item_code'].str.zfill(4)

KeyError: 'date'

In [115]:
df = df.groupby(['week_no', 'item_code']).sum(['holiday_flag', 'holiday_score', 'grow_score']).reset_index().sort_values(by='week_no', ascending=True)

In [102]:
df['item_code'] = df['item_code'].astype(str)
# 2자리 숫자로 패딩 (예: '1' -> '01')
df['item_code']  = df['item_code'].str.zfill(4)

In [116]:
df.drop_duplicates(inplace=True)
df.to_csv('test/holiday_작기_수정완료_utf-8_최종.csv', encoding='utf-8', index=False)

In [117]:
df

,week_no,item_code,holiday_flag,holiday_score,grow_score
0,2018년 01째주,0501,1,0.0,2.0
1,2018년 01째주,0601,1,0.0,0.0
2,2018년 01째주,1001,1,0.0,2.0
3,2018년 01째주,1005,1,0.0,6.0
4,2018년 01째주,1101,1,0.0,4.0
...,...,...,...,...,...
3146,2025년 28째주,1001,0,0.0,3.0
3147,2025년 28째주,1005,0,0.0,2.0
3148,2025년 28째주,1101,0,0.0,7.0
3149,2025년 28째주,1201,0,0.0,0.0


In [118]:
import os
from dotenv import load_dotenv
load_dotenv()

user = 'root'
password = os.getenv("DB_pw")
host = 'localhost'    # 또는 실제 호스트
port = 3306
db = 'jikfarm_db'
engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8mb4"
)
# 3. to_sql로 insert (테이블명, 커넥션, 옵션)
df.to_sql(
    name='factor_external_weekly',    # 실제 DB의 테이블명
    con=engine,
    if_exists='append',            # append: 추가 / replace: 전체 덮어쓰기
    index=False
)

print("DB 적재 완료!")

DB 적재 완료!


In [51]:
df_factor_external = pd.read_csv('test/holiday_작기_수정완료_utf-8.csv')

In [59]:
from datetime import datetime, date

def get_week_of_year(date):
    date = pd.to_datetime(date)
    year = date.year
    week_number = date.isocalendar().week
    return f"{year}년 {week_number}째주"

df['week_no'] = df['date'].apply(get_week_of_year)
df

,date,holiday_flag,holiday_score,grow_score,item_code,week_no
0,2018-01-01,1,0.0,1.0,1001,2018년 1째주
1,2018-01-02,0,0.0,1.0,1001,2018년 1째주
2,2018-01-03,0,0.0,1.0,1001,2018년 1째주
3,2018-01-04,0,0.0,1.0,1001,2018년 1째주
4,2018-01-05,0,0.0,1.0,1001,2018년 1째주
...,...,...,...,...,...,...
21987,2025-07-07,0,0.0,0.0,501,2025년 28째주
21988,2025-07-08,0,0.0,0.0,501,2025년 28째주
21989,2025-07-09,0,0.0,0.0,501,2025년 28째주
21990,2025-07-10,0,0.0,0.0,501,2025년 28째주


In [60]:
df.to_csv('test/holiday_작기_week_no_삽입.csv', encoding='utf-8', index=False)

In [62]:
df.drop(columns='date', inplace=True)
df.to_csv('test/holiday_작기_week_no_삽입_date제거.csv', encoding='utf-8', index=False)


In [63]:
df.to_csv('test/holiday_작기_week_no_삽입_date제거_cp949.csv', encoding='cp949', index=False)

In [18]:
df_master_crop_variety = pd.read_csv('표준코드/표준코드_품목코드_utf8.csv')
df_master_crop_variety.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17555 entries, 0 to 17554
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   gds_lclsf_cd  17555 non-null  object
 1   gds_lclsf_nm  17555 non-null  object
 2   gds_mclsf_cd  17555 non-null  object
 3   gds_mclsf_nm  17555 non-null  object
 4   gds_sclsf_cd  17555 non-null  object
 5   gds_sclsf_nm  17555 non-null  object
dtypes: object(6)
memory usage: 823.0+ KB


In [28]:
def create_crop_codes(df: pd.DataFrame) -> pd.DataFrame:
    """
    주어진 DataFrame에 'item_code'와 'full_code' 컬럼을 생성합니다.

    'item_code'는 'lclsf_cd'와 'mclsf_cd'를 조합하여 생성됩니다.
    'full_code'는 'lclsf_cd', 'mclsf_cd', 'sclsf_cd'를 조합하여 생성됩니다.

    Args:
        df (pd.DataFrame): 'lclsf_cd', 'mclsf_cd', 'sclsf_cd' 컬럼을 포함하는 DataFrame.

    Returns:
        pd.DataFrame: 'item_code'와 'full_code' 컬럼이 추가된 DataFrame.
    """
    # 각 코드 컬럼이 문자열 타입인지 확인하고, 아니면 문자열로 변환합니다.
    # 이는 코드들이 숫자형으로 읽혔을 때 발생할 수 있는 오류를 방지합니다.
    for col in ['gds_lclsf_cd', 'gds_mclsf_cd', 'gds_sclsf_cd']:
        if col in df.columns:
            df[col] = df[col].astype(str)
            # 2자리 숫자로 패딩 (예: '1' -> '01')
            df[col] = df[col].str.zfill(2)
        else:
            print(f"경고: '{col}' 컬럼이 DataFrame에 없습니다. 코드 생성에 문제가 있을 수 있습니다.")
            return df # 필수 컬럼이 없으면 함수 종료

    # 'item_code' 생성: 대분류 코드 + 품목 코드
    # 예: '11' + '01' = '1101'
    df['item_code'] = df['gds_lclsf_cd'] + df['gds_mclsf_cd']

    # 'full_code' 생성: 대분류 코드 + 품목 코드 + 품종 코드
    # 예: '11' + '01' + '01' = '110101'
    df['full_code'] = df['gds_lclsf_cd'] + df['gds_mclsf_cd'] + df['gds_sclsf_cd']

    return df

In [29]:
df = create_crop_codes(df_master_crop_variety)
df

NameError: name 'df_master_crop_variety' is not defined

In [31]:
df.to_csv('test/표준코드_품목코드_utf8_sig_2.csv', encoding='utf-8-sig', index=False)

# 품목코드 DB 삽입

In [4]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

In [22]:
df.to_csv('test/표준코드_품목코드_cp.csv', encoding='cp949')

In [30]:
df = pd.read_csv('test/표준코드_품목코드_cp.csv', encoding='cp949')
create_crop_codes(df)

,gds_lclsf_cd,gds_lclsf_nm,gds_mclsf_cd,gds_mclsf_nm,gds_sclsf_cd,gds_sclsf_nm,item_code,full_code
0,01,미곡류,00,미곡류(일반),00,-,0100,010000
1,01,미곡류,01,벼,00,벼,0101,010100
2,01,미곡류,01,벼,01,일반계,0101,010101
3,01,미곡류,01,벼,02,통일계,0101,010102
4,01,미곡류,01,벼,03,햇일반계,0101,010103
...,...,...,...,...,...,...,...,...
2066,14,산채류,99,기타,14,삼잎국화나물,1499,149914
2067,14,산채류,99,기타,15,어수리나물,1499,149915
2068,14,산채류,99,기타,16,삿갓나물,1499,149916
2069,14,산채류,99,기타,17,병풍취,1499,149917


In [31]:
df.drop_duplicates(inplace=True)
df.to_csv('test/표준코드_품목코드_cp2.csv', encoding='cp949')

In [32]:
df

,gds_lclsf_cd,gds_lclsf_nm,gds_mclsf_cd,gds_mclsf_nm,gds_sclsf_cd,gds_sclsf_nm,item_code,full_code
0,01,미곡류,00,미곡류(일반),00,-,0100,010000
1,01,미곡류,01,벼,00,벼,0101,010100
2,01,미곡류,01,벼,01,일반계,0101,010101
3,01,미곡류,01,벼,02,통일계,0101,010102
4,01,미곡류,01,벼,03,햇일반계,0101,010103
...,...,...,...,...,...,...,...,...
2066,14,산채류,99,기타,14,삼잎국화나물,1499,149914
2067,14,산채류,99,기타,15,어수리나물,1499,149915
2068,14,산채류,99,기타,16,삿갓나물,1499,149916
2069,14,산채류,99,기타,17,병풍취,1499,149917


In [21]:
df[df['full_code']=='19O102']

,gds_lclsf_cd,gds_lclsf_nm,gds_mclsf_cd,gds_mclsf_nm,gds_sclsf_cd,gds_sclsf_nm,item_code,full_code
3061,19,약용작물류,O1,인삼,02,수경인삼,19O1,19O102


In [33]:
import os
from dotenv import load_dotenv
load_dotenv()

user = 'root'
password = os.getenv("DB_pw")
host = 'localhost'    # 또는 실제 호스트
port = 3306
db = 'jikfarm_db'
engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8mb4"
)
# 3. to_sql로 insert (테이블명, 커넥션, 옵션)
df.to_sql(
    name='master_crop_variety',    # 실제 DB의 테이블명
    con=engine,
    if_exists='append',            # append: 추가 / replace: 전체 덮어쓰기
    index=False
)

print("DB 적재 완료!")

DB 적재 완료!


# 산지코드-직팜코드-관측소코드 매핑

In [34]:
import pandas as pd
df = pd.read_csv('test/산지코드_직팜_관측지점_매핑완료_수정.csv', encoding='cp949')

In [35]:
def expand_row(row):
    if '~' in str(row['산지코드']):
        start, end = map(int, row['산지코드'].split('~'))
        return [
            {**row, '산지코드': code}
            for code in range(start, end+1)
        ]
    else:
        return [{**row, '산지코드': int(row['산지코드'])}]

# 예시 DataFrame: df
expanded = []
for _, row in df.iterrows():
    expanded.extend(expand_row(row))

df_expanded = pd.DataFrame(expanded)
df_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800035 entries, 0 to 800034
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   산지코드    800035 non-null  int64  
 1   산지이름    800035 non-null  object 
 2   직팜산지코드  800035 non-null  int64  
 3   직팜산지이름  800035 non-null  object 
 4   관측지점    700034 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 30.5+ MB


In [39]:
df = df_expanded.rename(columns={'산지코드' : 'plor_cd',
                            '산지이름': 'plor_nm',
                            '직팜산지코드': 'j_sanji_cd', 
                            '직팜산지이름': 'j_sanji_nm', 
                            '관측지점' : 'station_cd'
                           })
df

,plor_cd,plor_nm,j_sanji_cd,j_sanji_nm,station_cd
0,100000,서울특별시,1000,서울특별시,108.0
1,100001,서울특별시,1000,서울특별시,108.0
2,100002,서울특별시,1000,서울특별시,108.0
3,100003,서울특별시,1000,서울특별시,108.0
4,100004,서울특별시,1000,서울특별시,108.0
...,...,...,...,...,...
800030,971000,경상남도 창원시,1120,경상남도 창원시,255.0
800031,980000,경상북도,1169,경상북도,136.0
800032,981000,경상북도 포항시,1159,경상북도 포항시,138.0
800033,990000,제주도,1170,제주특별자치도,184.0


In [40]:
df.to_csv('test/산지코드_직팜_관측지점_매핑완료_수정_utf-8_2.csv', encoding='utf-8', index=False)

In [41]:
import os
from dotenv import load_dotenv
load_dotenv()

user = 'root'
password = os.getenv("DB_pw")
host = 'localhost'    # 또는 실제 호스트
port = 3306
db = 'jikfarm_db'
engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8mb4"
)
# 3. to_sql로 insert (테이블명, 커넥션, 옵션)
df.to_sql(
    name='map_region_weather_station',    # 실제 DB의 테이블명
    con=engine,
    if_exists='append',            # append: 추가 / replace: 전체 덮어쓰기
    index=False
)

print("DB 적재 완료!")

DB 적재 완료!


# 일별 기상 데이터 

In [42]:
df = pd.read_csv('test/기상청_서울_일기요소_20180101-20250531.csv', encoding='cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257907 entries, 0 to 257906
Data columns (total 56 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TM             257907 non-null  int64  
 1   STN            257907 non-null  int64  
 2   WS_AVG         257907 non-null  float64
 3   WR_DAY         257907 non-null  int64  
 4   WD_MAX         257907 non-null  int64  
 5   WS_MAX         257907 non-null  float64
 6   WS_MAX_TM      257907 non-null  int64  
 7   WD_INS         257907 non-null  int64  
 8   WS_INS         257907 non-null  float64
 9   WS_INS_TM      257907 non-null  int64  
 10  TA_AVG         257907 non-null  float64
 11  TA_MAX         257907 non-null  float64
 12  TA_MAX_TM      257907 non-null  int64  
 13  TA_MIN         257907 non-null  float64
 14  TA_MIN_TM      257907 non-null  int64  
 15  TD_AVG         257907 non-null  float64
 16  TS_AVG         257907 non-null  float64
 17  TG_MIN         257907 non-nul

In [45]:
df= df.loc[:, ['TM', 'STN', 'TA_AVG', 'TA_MAX', 'TA_MIN', 'HM_AVG', 'RN_DAY', 'RN_60M_MAX']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257907 entries, 0 to 257906
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   TM          257907 non-null  int64  
 1   STN         257907 non-null  int64  
 2   TA_AVG      257907 non-null  float64
 3   TA_MAX      257907 non-null  float64
 4   TA_MIN      257907 non-null  float64
 5   HM_AVG      257907 non-null  float64
 6   RN_DAY      257907 non-null  float64
 7   RN_60M_MAX  257907 non-null  float64
dtypes: float64(6), int64(2)
memory usage: 15.7 MB


In [46]:
import os
from dotenv import load_dotenv
load_dotenv()

user = 'root'
password = os.getenv("DB_pw")
host = 'localhost'    # 또는 실제 호스트
port = 3306
db = 'jikfarm_db'
engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8mb4"
)
# 3. to_sql로 insert (테이블명, 커넥션, 옵션)
df.to_sql(
    name='weather_daily',    # 실제 DB의 테이블명
    con=engine,
    if_exists='append',            # append: 추가 / replace: 전체 덮어쓰기
    index=False
)

print("DB 적재 완료!")

DB 적재 완료!
